In [12]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [2]:
max_dt = pd.to_datetime("1900-01-01 12:00:00")
min_dt = pd.to_datetime("2099-01-01 12:00:00")
csv_dict = {
    "basal" : pd.read_csv("IT 30 ENG/Insulin data/basal_data_1.csv",skiprows=1)
    ,"bolus" : pd.read_csv("IT 30 ENG/Insulin data/bolus_data_1.csv",skiprows=1)
    ,"insulin" : pd.read_csv("IT 30 ENG/Insulin data/insulin_data_1.csv",skiprows=1)
    ,"alarms": pd.read_csv("IT 30 ENG/alarms_data_1.csv",skiprows=1)
    ,"bg" : pd.read_csv("IT 30 ENG/bg_data_1.csv",skiprows=1)
    ,"cgm" : pd.read_csv("IT 30 ENG/cgm_data_1.csv",skiprows=1)
}

for key, value in csv_dict.items():
    value["Timestamp"] = pd.to_datetime(value["Timestamp"])
    value = value.set_index("Timestamp").drop("Serial Number", axis=1)
    value = value.add_prefix(f"{key}_")
    if max_dt < value.index.max():
        max_dt = value.index.max()
    if min_dt > value.index.min():
        min_dt = value.index.min()
    csv_dict[key] = value

In [20]:
df = pd.DataFrame({'DateTime':pd.date_range(start=min_dt, end=max_dt, freq='1min')})
df = df.set_index('DateTime')

for key, value in csv_dict.items():
    df = df.join(value)
display(df)

#df.to_csv("PumpPerfect_compiled.csv")

# df[~df['this_column'].isna()]
# df.query('A > B')

,basal_Insulin Type,basal_Duration (minutes),basal_Percentage (%),basal_Rate,basal_Insulin Delivered (U),bolus_Insulin Type,bolus_Blood Glucose Input (mmol/l),bolus_Carbs Input (g),bolus_Carbs Ratio,bolus_Insulin Delivered (U),bolus_Initial Delivery (U),bolus_Extended Delivery (U),insulin_Total Bolus (U),insulin_Total Insulin (U),insulin_Total Basal (U),alarms_Alarm/Event,bg_Glucose Value (mmol/l),bg_Manual Reading,cgm_CGM Glucose Value (mmol/l)
DateTime,,,,,,,,,,,,,,,,,,,
2024-06-02 00:00:00,Scheduled,1.0,NaN,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-02 00:01:00,Scheduled,5.0,NaN,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
2024-06-02 00:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-02 00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-02 00:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-01 21:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.64,15.23,11.59,NaN,NaN,NaN,NaN
2024-07-01 21:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-01 21:52:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df["basal_Insulin Delivered (U)"] = df["basal_Rate"]/60 # df["basal_Duration (minutes)"] *
df["basal_Insulin Delivered (U)"] = df["basal_Insulin Delivered (U)"].ffill()
df['bolus_Trigger'] = 0
df['bolus_Trigger'].loc[(df['bolus_Carbs Input (g)'] == 0)] = 1 # automatický bolus
df['bolus_Trigger'].loc[(df['bolus_Carbs Input (g)'] != 0) & (df['bolus_Carbs Input (g)'].notna()) & (df['bolus_Carbs Ratio'] != 0) & (df['bolus_Carbs Ratio'].notna())] = 2 # manuální bolus s jídlem
df['bolus_Trigger'].loc[(df['bolus_Carbs Input (g)'] == 0) & (df['bolus_Carbs Input (g)'].notna()) & (df['bolus_Carbs Ratio'] != 0) & (df['bolus_Carbs Ratio'].notna())] = 3 # manuální bolus bez jídla
df["bolus_Insulin Delivered (U)"] = df["bolus_Insulin Delivered (U)"].fillna(0)
df["cgm_CGM Glucose Value (mmol/l)"] = df["cgm_CGM Glucose Value (mmol/l)"].interpolate(method='time')
df["HypoEvent"] = 0
time_window = 120
for idx, row in df.reset_index().iterrows():
    if row["bolus_Trigger"] != 0:
        if any(df["cgm_CGM Glucose Value (mmol/l)"][idx:idx+time_window] < 3.9):
            df["HypoEvent"][idx] = 1
df["Time"] = (df.index).time
display(df) 

C:\Users\Ondre\AppData\Local\Temp\ipykernel_17620\1306445647.py:5: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


C:\Users\Ondre\AppData\Local\Temp\ipykernel_17620\1306445647.py:5: SettingWithCopyWarning:


A value is trying to be set on a co

,basal_Insulin Type,basal_Duration (minutes),basal_Percentage (%),basal_Rate,basal_Insulin Delivered (U),bolus_Insulin Type,bolus_Blood Glucose Input (mmol/l),bolus_Carbs Input (g),bolus_Carbs Ratio,bolus_Insulin Delivered (U),...,insulin_Total Bolus (U),insulin_Total Insulin (U),insulin_Total Basal (U),alarms_Alarm/Event,bg_Glucose Value (mmol/l),bg_Manual Reading,cgm_CGM Glucose Value (mmol/l),bolus_Trigger,HypoEvent,Time
DateTime,,,,,,,,,,,,,,,,,,,,,
2024-06-02 00:00:00,Scheduled,1.0,NaN,0.4,0.006667,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,00:00:00
2024-06-02 00:01:00,Scheduled,5.0,NaN,0.4,0.006667,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.00,0,0,00:01:00
2024-06-02 00:02:00,NaN,NaN,NaN,NaN,0.006667,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,9.96,0,0,00:02:00
2024-06-02 00:03:00,NaN,NaN,NaN,NaN,0.006667,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,9.92,0,0,00:03:00
2024-06-02 00:04:00,NaN,NaN,NaN,NaN,0.006667,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,9.88,0,0,00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-01 21:50:00,NaN,NaN,NaN,NaN,0.013383,NaN,NaN,NaN,NaN,0.0,...,3.64,15.23,11.59,NaN,NaN,NaN,10.50,0,0,21:50:00
2024-07-01 21:51:00,NaN,NaN,NaN,NaN,0.013383,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.50,0,0,21:51:00
2024-07-01 21:52:00,NaN,NaN,NaN,NaN,0.013383,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.50,0,0,21:52:00


In [ ]:
print(df.columns)

# Select the ones you want
df = df[['basal_Insulin Delivered (U)','bolus_Insulin Delivered (U)', 'bolus_Trigger', 'cgm_CGM Glucose Value (mmol/l)']]
df.to_csv()

Index(['basal_Insulin Type', 'basal_Duration (minutes)',
       'basal_Percentage (%)', 'basal_Rate', 'basal_Insulin Delivered (U)',
       'bolus_Insulin Type', 'bolus_Blood Glucose Input (mmol/l)',
       'bolus_Carbs Input (g)', 'bolus_Carbs Ratio',
       'bolus_Insulin Delivered (U)', 'bolus_Initial Delivery (U)',
       'bolus_Extended Delivery (U)', 'insulin_Total Bolus (U)',
       'insulin_Total Insulin (U)', 'insulin_Total Basal (U)',
       'alarms_Alarm/Event', 'bg_Glucose Value (mmol/l)', 'bg_Manual Reading',
       'cgm_CGM Glucose Value (mmol/l)', 'bolus_Trigger', 'Time'],
      dtype='object')


In [7]:
# Initialize the Dash app
app = Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Time Series Line Plot"),
    
    # Date picker for selecting date range (optional)
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df.index.min(),
        end_date=df.index.max()
    ),
    
    # Line plot
    dcc.Graph(id='line-plot')
])

# Callback to update the line plot based on date range selection
@app.callback(
    Output('line-plot', 'figure'),
    Input('date-range-picker', 'start_date'),
    Input('date-range-picker', 'end_date')
)
def update_line_plot(start_date, end_date):
    # Filter data based on the selected date range
    filtered_df = df.loc[start_date:end_date]
    
    # Create line plot with Plotly Express
    fig = px.line(
        filtered_df, 
        x=filtered_df.index, 
        y='basal_Insulin Delivered (U)', 
        title="Time Series Line Plot",
        labels={"x": "Date", "Value": "Value"}
    )
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Value"
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [8]:
# Specify the series to plot directly in the code
selected_series = ['basal_Insulin Delivered (U)', 'cgm_CGM Glucose Value (mmol/l)']  # Choose specific columns to plot

#'basal_Insulin Delivered (U)','bolus_Insulin Delivered (U)', 'bolus_Trigger', 'cgm_CGM Glucose Value (mmol/l)'

# Initialize the Dash app
app = Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Pre-Selected Time Series Line Plot"),
    
    # Date picker for selecting date range
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df.index.min(),
        end_date=df.index.max()
    ),
    
    # Line plot
    dcc.Graph(id='line-plot')
])

# Callback to update the line plot based on selected date range
@app.callback(
    Output('line-plot', 'figure'),
    Input('date-range-picker', 'start_date'),
    Input('date-range-picker', 'end_date')
)
def update_line_plot(start_date, end_date):
    # Filter data based on selected date range and specific series
    filtered_df = df.loc[start_date:end_date, selected_series]
    
    # Reshape data for Plotly Express compatibility
    filtered_df_long = filtered_df.reset_index().melt(id_vars='DateTime', var_name='Series', value_name='Value')
    
    # Create a line plot with Plotly Express
    fig = px.line(
        filtered_df_long, 
        x='DateTime', 
        y='Value', 
        color='Series', 
        title="Time Series Line Plot with Pre-Selected Series",
        labels={"index": "Date", "Value": "Value"}
    )
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Value"
    )
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


In [136]:
# Specify the series to plot in each figure
selected_series = {
    "Figure 1": ["basal_Insulin Delivered (U)"],  # First figure with two series
    "Figure 2": ["bolus_Insulin Delivered (U)"],             # Second figure with one series
    "Figure 3": ["cgm_CGM Glucose Value (mmol/l)"]    # Third figure with another combination
}

#'basal_Insulin Delivered (U)','bolus_Insulin Delivered (U)', 'bolus_Trigger', 'cgm_CGM Glucose Value (mmol/l)'

# Initialize the Dash app
app = Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Multiple Line Plots with Shared Date Picker"),
    
    # Date picker for selecting date range
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df.index.min(),
        end_date=df.index.max()
    ),
    
    # Multiple line plots
    html.Div([
        dcc.Graph(id='line-plot-1'),
        dcc.Graph(id='line-plot-2'),
        dcc.Graph(id='line-plot-3')
    ])
])

# Callback to update each line plot based on selected date range
@app.callback(
    [Output('line-plot-1', 'figure'),
     Output('line-plot-2', 'figure'),
     Output('line-plot-3', 'figure')],
    [Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)

def update_line_plots(start_date, end_date):
    # Create figures for each plot
    figures = []
    for fig_title, series_list in selected_series.items():
        # Filter data for the specified date range and series list
        filtered_df = df.loc[start_date:end_date, series_list]
        
        # Reshape data to long format for Plotly Express
        filtered_df_long = filtered_df.reset_index().melt(id_vars='DateTime', var_name='Series', value_name='Value')
        
        # Create a line plot
        fig = px.line(
            filtered_df_long,
            x='DateTime',
            y='Value',
            color='Series',
            title=fig_title,
            labels={"index": "Date", "Value": "Value"}
        )
        
        fig.update_layout(
            xaxis_title="Date",
            yaxis_title="Value"
        )
        
        # Add the figure to the list
        figures.append(fig)
    
    return figures

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


In [137]:
# Specify the series to plot directly in the code
selected_series = ['basal_Insulin Delivered (U)', 'bolus_Insulin Delivered (U)']  # Choose specific columns to plot

#'basal_Insulin Delivered (U)','bolus_Insulin Delivered (U)', 'bolus_Trigger', 'cgm_CGM Glucose Value (mmol/l)'

# Initialize the Dash app
app = Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Pre-Selected Time Series Line Plot"),
    
    # Date picker for selecting date range
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df.index.min(),
        end_date=df.index.max()
    ),
    
    # Line plot
    dcc.Graph(id='line-plot')
])

# Callback to update the line plot based on selected date range
@app.callback(
    Output('line-plot', 'figure'),
    Input('date-range-picker', 'start_date'),
    Input('date-range-picker', 'end_date')
)
def update_line_plot(start_date, end_date):
    # Filter data based on selected date range and specific series
    filtered_df = df.loc[start_date:end_date, selected_series]
    
    # Reshape data for Plotly Express compatibility
    filtered_df_long = filtered_df.reset_index().melt(id_vars='DateTime', var_name='Series', value_name='Value')

    filtered_df_long = filtered_df_long.set_index("DateTime").join(df['bolus_Trigger'])


    # Add a color column based on the bolus_Trigger condition
    filtered_df_long['Color'] = np.where(
        (filtered_df_long['Series'] == 'bolus_Insulin Delivered (U)') & (filtered_df_long['bolus_Trigger'].values == "Automatic"),
        'red', 'blue'
    )

    filtered_df_long = filtered_df_long.drop("bolus_Trigger", axis=1).reset_index()
    display(filtered_df_long[filtered_df_long["Color"] == 'red'])
    
    # Create a line plot with Plotly Express
    fig = px.line(
        filtered_df_long, 
        x='DateTime', 
        y='Value', 
        color='Series', 
        title="Time Series Line Plot with Pre-Selected Series",
        labels={"index": "Date", "Value": "Value"},
        color_discrete_sequence=filtered_df_long['Color']  # Use the conditional colors
    )
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Value"
    )
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


,DateTime,Series,Value,Color


,DateTime,Series,Value,Color


,DateTime,Series,Value,Color


,DateTime,Series,Value,Color


In [6]:
def SummarizeMetric(df, col):
    df_summary = df.groupby('Time').agg(
        Median=(col, 'median'),
        Q1=(col, lambda x: x.quantile(0.25)),
        Q3=(col, lambda x: x.quantile(0.75)),
        D1=(col, lambda x: x.quantile(0.1)),
        D9=(col, lambda x: x.quantile(0.9)),
        Min=(col, 'min'),
        Max=(col, 'max'),
    ).reset_index()
    
    return df_summary

In [7]:
df["Time"] = (df.index).time

In [ ]:
df_glucose = SummarizeMetric(df, 'cgm_CGM Glucose Value (mmol/l)')
display(df_glucose)

In [ ]:
import plotly.graph_objects as go

# Initialize the Dash app
app = Dash(__name__)

# Create a line plot using Plotly
def create_plot():
    fig = go.Figure()

    # Add median line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['Median'],
        mode='lines',
        name='Median',
        line=dict(color='green')
    ))

    # Add first quartile (Q1) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['Q1'],
        mode='lines',
        name='Q1 and Q3',
        line=dict(color='orange', dash='dot'),
        showlegend=False  # Hide legend for this trace
    ))

    # Add third quartile (Q3) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['Q3'],
        mode='lines',
        line=dict(color='orange', dash='dot'),
        fill='tonexty',  # This fills the area between the Q1 and Q3 lines
        fillcolor='rgba(255, 165, 0, 0.3)',  # Set fill color with transparency
        name='Q1 and Q3'  # Use same label for legend
    ))

    # Add first quartile (D1) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['D1'],
        mode='lines',
        name='D1 and D9',
        line=dict(color='blue', dash='dot'),
        showlegend=False  # Hide legend for this trace
    ))

    # Add third quartile (D9) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['D9'],
        mode='lines',
        line=dict(color='blue', dash='dot'),
        fill='tonexty',  # This fills the area between the Q1 and Q3 lines
        fillcolor='rgba(0, 165, 255, 0.1)',  # Set fill color with transparency
        name='D1 and D9'  # Use same label for legend
    ))

    # Add first quartile (D1) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['Max'],
        mode='lines',
        name='Min and Max',
        line=dict(color='yellow', dash='dash'),
        showlegend=False  # Hide legend for this trace,
    ))

    # Add third quartile (D9) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['Min'],
        mode='lines',
        line=dict(color='yellow', dash='dash'),
        name='Min and Max'  # Use same label for legend
    ))

    # Update layout
    fig.update_layout(
        title='Summary Statistics by Time of Day',
        xaxis_title='Time of Day',
        yaxis_title='Value',
        xaxis=dict(tickmode='array'),
        legend=dict(x=0.01, y=0.99)
    )

    return fig

# Define the layout of the app
app.layout = html.Div([
    html.H1("Time Series Summary Statistics"),
    dcc.Graph(figure=create_plot())  # Use the create_plot function to generate the figure
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [8]:
df_basal_insulin = SummarizeMetric(df, 'basal_Insulin Delivered (U)')
display(df_basal_insulin)

,Time,Median,Q1,Q3,D1,D9,Min,Max
0,00:00:00,0.006667,0.006667,0.006875,0.006667,0.013658,0.002533,0.018817
1,00:01:00,0.006667,0.006667,0.006667,0.006253,0.013292,0.000000,0.018817
2,00:02:00,0.006667,0.006667,0.007500,0.006357,0.013907,0.000000,0.018817
3,00:03:00,0.007083,0.006667,0.007500,0.006357,0.016892,0.000000,0.019450
4,00:04:00,0.007500,0.006667,0.007500,0.006357,0.016892,0.000000,0.019450
...,...,...,...,...,...,...,...,...
1435,23:55:00,0.006667,0.006667,0.006667,0.006667,0.011453,0.000000,0.018817
1436,23:56:00,0.006667,0.006667,0.006667,0.006667,0.011453,0.000000,0.018817
1437,23:57:00,0.006667,0.006667,0.006667,0.006667,0.011453,0.002533,0.018817
1438,23:58:00,0.006667,0.006667,0.006667,0.006667,0.014163,0.002533,0.018817


In [9]:
df_bolus_insulin = SummarizeMetric(df, 'bolus_Insulin Delivered (U)')
display(df_bolus_insulin)

,Time,Median,Q1,Q3,D1,D9,Min,Max
0,00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,1.45
1,00:01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,00:02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,00:03:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4,00:04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...
1435,23:55:00,0.0,0.0,0.0,0.0,0.0,0.0,1.10
1436,23:56:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1437,23:57:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1438,23:58:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [15]:
df_autoXmanual = df.groupby("Time")["bolus_Trigger"].max()
colors = replaced_array = np.where(df_autoXmanual.values == 0, 'white',
                    np.where(df_autoXmanual.values == 1, 'blue',
                    np.where(df_autoXmanual.values == 2, 'purple',
                    np.where(df_autoXmanual.values == 3, 'red', df_autoXmanual.values))))

# Initialize the Dash app
app = Dash(__name__)

# Create a line plot using Plotly
def create_plot():
    fig = go.Figure()

    # Add median line
    fig.add_trace(go.Scatter(
        x=df_basal_insulin['Time'],
        y=df_basal_insulin['Median'],
        mode='lines',
        name='Median',
        line=dict(color='green')
    ))

    # Add first quartile (Q1) line
    fig.add_trace(go.Scatter(
        x=df_basal_insulin['Time'],
        y=df_basal_insulin['Q1'],
        mode='lines',
        name='Q1 and Q3',
        line=dict(color='orange', dash='dot'),
        showlegend=False  # Hide legend for this trace
    ))

    # Add third quartile (Q3) line
    fig.add_trace(go.Scatter(
        x=df_basal_insulin['Time'],
        y=df_basal_insulin['Q3'],
        mode='lines',
        line=dict(color='orange', dash='dot'),
        fill='tonexty',  # This fills the area between the Q1 and Q3 lines
        fillcolor='rgba(255, 165, 0, 0.3)',  # Set fill color with transparency
        name='Q1 and Q3'  # Use same label for legend
    ))

        # Segment bolus line based on threshold, but hide individual segments from legend
    for i in range(len(df_bolus_insulin) - 1):
        x_segment = [df_bolus_insulin['Time'].iloc[i], df_bolus_insulin['Time'].iloc[i + 1]]
        y_segment = [df_bolus_insulin['Max'].iloc[i], df_bolus_insulin['Max'].iloc[i + 1]]
        
        # Choose color based on threshold
        #color = 'red' if df_bolus_insulin['Max'].iloc[i] >= threshold else 'blue'
        color = colors[i]
        
        # Add segment as a separate trace, without showing it in the legend
        fig.add_trace(go.Scatter(
            x=x_segment,
            y=y_segment,
            mode='lines+markers',
            yaxis="y2",
            line=dict(color=color, width=2),
            showlegend=False,
            marker=dict(
                symbol='circle',  # Choose marker shape
                size=8,           # Set marker size
                color=color,      # Set marker color
                opacity=0.8,      # Set marker opacity
            )
        ))

    # Add a "dummy" trace for the legend
    fig.add_trace(go.Scatter(
        x=[None],  # No actual points
        y=[None],
        mode='lines',
        name='Bolus Insulin',
        line=dict(color='blue')  # Use one of the colors to represent the legend item
    ))

    # Update layout
    fig.update_layout(
        title='Summary Statistics by Time of Day',
        xaxis_title='Time of Day',
        yaxis_title='Basal',
        yaxis2=dict(
            title='Bolus',
            overlaying='y',
            side='right',
            showgrid=False  # Hide grid for secondary y-axis if desired
        ),
        xaxis=dict(tickmode='array'),
        legend=dict(x=0.01, y=0.99)
    )

    return fig

# Define the layout of the app
app.layout = html.Div([
    html.H1("Time Series Summary Statistics"),
    dcc.Graph(figure=create_plot())  # Use the create_plot function to generate the figure
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [25]:
df_autoXmanual = df.groupby("Time")["bolus_Trigger"].max()
colors = replaced_array = np.where(df_autoXmanual.values == 0, 'white',
                    np.where(df_autoXmanual.values == 1, 'blue',
                    np.where(df_autoXmanual.values == 2, 'purple',
                    np.where(df_autoXmanual.values == 3, 'red', df_autoXmanual.values))))

# Initialize the Dash app
app = Dash(__name__)

# Create a line plot using Plotly
def create_plot():
    fig = go.Figure()

    # Add median line
    fig.add_trace(go.Scatter(
        x=df_basal_insulin['Time'],
        y=df_basal_insulin['Median'],
        mode='lines',
        name='Median',
        line=dict(color='green')
    ))

    # Add first quartile (Q1) line
    fig.add_trace(go.Scatter(
        x=df_basal_insulin['Time'],
        y=df_basal_insulin['Q1'],
        mode='lines',
        name='Q1 and Q3',
        line=dict(color='orange', dash='dot'),
        showlegend=False  # Hide legend for this trace
    ))

    # Add third quartile (Q3) line
    fig.add_trace(go.Scatter(
        x=df_basal_insulin['Time'],
        y=df_basal_insulin['Q3'],
        mode='lines',
        line=dict(color='orange', dash='dot'),
        fill='tonexty',  # This fills the area between the Q1 and Q3 lines
        fillcolor='rgba(255, 165, 0, 0.3)',  # Set fill color with transparency
        name='Q1 and Q3'  # Use same label for legend
    ))

        # Segment bolus line based on threshold, but hide individual segments from legend
    for i in range(len(df_bolus_insulin) - 1):
        x_segment = [df_bolus_insulin['Time'].iloc[i], df_bolus_insulin['Time'].iloc[i + 1]]
        y_segment = [df_bolus_insulin['Max'].iloc[i], df_bolus_insulin['Max'].iloc[i + 1]]
        
        # Choose color based on threshold
        #color = 'red' if df_bolus_insulin['Max'].iloc[i] >= threshold else 'blue'
        color = colors[i]
        
        # Add segment as a separate trace, without showing it in the legend
        fig.add_trace(go.Scatter(
            x=x_segment,
            y=y_segment,
            mode='lines+markers',
            yaxis="y2",
            line=dict(color=color, width=2),
            showlegend=False,
            marker=dict(
                symbol='circle',  # Choose marker shape
                size=8,           # Set marker size
                color=color,      # Set marker color
                opacity=0.8,      # Set marker opacity
            )
        ))

    # Add a "dummy" trace for the legend
    fig.add_trace(go.Scatter(
        x=[None],  # No actual points
        y=[None],
        mode='lines',
        name='Bolus Insulin',
        line=dict(color='blue')  # Use one of the colors to represent the legend item
    ))

    # Update layout
    fig.update_layout(
        title='Summary Statistics by Time of Day',
        xaxis_title='Time of Day',
        yaxis_title='Basal',
        yaxis2=dict(
            title='Bolus',
            overlaying='y',
            side='right',
            showgrid=False  # Hide grid for secondary y-axis if desired
        ),
        xaxis=dict(tickmode='array'),
        legend=dict(x=0.01, y=0.99)
    )

    return fig

# Define the layout of the app
app.layout = html.Div([
    html.H1("Time Series Summary Statistics"),
    dcc.Graph(figure=create_plot())  # Use the create_plot function to generate the figure
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [133]:
display(df[df["HypoEvent"] != 0])

,basal_Insulin Type,basal_Duration (minutes),basal_Percentage (%),basal_Rate,basal_Insulin Delivered (U),bolus_Insulin Type,bolus_Blood Glucose Input (mmol/l),bolus_Carbs Input (g),bolus_Carbs Ratio,bolus_Insulin Delivered (U),...,insulin_Total Bolus (U),insulin_Total Insulin (U),insulin_Total Basal (U),alarms_Alarm/Event,bg_Glucose Value (mmol/l),bg_Manual Reading,cgm_CGM Glucose Value (mmol/l),bolus_Trigger,HypoEvent,Time
DateTime,,,,,,,,,,,,,,,,,,,,,
2024-06-04 15:34:00,NaN,NaN,NaN,NaN,0.010000,Normal,9.8,35.0,20.0,1.750,...,NaN,NaN,NaN,NaN,9.8,M,9.72,2,1,15:34:00
2024-06-04 15:57:00,NaN,NaN,NaN,NaN,0.010000,Normal,10.1,42.0,20.0,2.100,...,NaN,NaN,NaN,NaN,10.1,M,10.02,2,1,15:57:00
2024-06-07 10:49:00,Scheduled,5.0,NaN,1.090,0.018167,Normal,10.5,0.0,NaN,0.409,...,NaN,NaN,NaN,NaN,NaN,NaN,8.92,1,1,10:49:00
2024-06-07 11:27:00,NaN,NaN,NaN,NaN,0.008333,Normal,9.6,28.0,20.0,1.490,...,NaN,NaN,NaN,NaN,NaN,NaN,9.48,2,1,11:27:00
2024-06-10 09:28:00,NaN,NaN,NaN,NaN,0.005850,Normal,6.0,33.0,20.0,1.630,...,NaN,NaN,NaN,NaN,NaN,NaN,5.94,2,1,09:28:00
2024-06-11 09:35:00,Scheduled,5.0,NaN,1.043,0.017383,Normal,10.2,0.0,NaN,0.380,...,NaN,NaN,NaN,NaN,NaN,NaN,6.40,1,1,09:35:00
2024-06-14 10:01:00,NaN,NaN,NaN,NaN,0.008333,Normal,9.8,26.0,20.0,1.300,...,NaN,NaN,NaN,NaN,NaN,NaN,9.80,2,1,10:01:00
2024-06-15 11:28:00,NaN,NaN,NaN,NaN,0.008333,Normal,10.0,0.0,NaN,0.165,...,NaN,NaN,NaN,NaN,NaN,NaN,8.64,1,1,11:28:00
2024-06-15 11:29:00,NaN,NaN,NaN,NaN,0.008333,Normal,8.7,41.0,20.0,2.050,...,NaN,NaN,NaN,NaN,8.7,M,8.70,2,1,11:29:00


In [24]:
# Initialize the Dash app
app = Dash(__name__)

# Create a line plot using Plotly
def create_plot():
    fig = go.Figure()

    # # Segment bolus line based on threshold, but hide individual segments from legend
    # for i in range(len(df_bolus_insulin) - 1):
    #     x_segment = [df['DateTime'].iloc[i], df_bolus_insulin['Time'].iloc[i + 1]]
    #     y_segment = [df['Max'].iloc[i], df_bolus_insulin['Max'].iloc[i + 1]]
        
    #     # Choose color based on threshold
    #     #color = 'red' if df_bolus_insulin['Max'].iloc[i] >= threshold else 'blue'
    #     color = colors[i]
        
    #     # Add segment as a separate trace, without showing it in the legend
    #     fig.add_trace(go.Scatter(
    #         x=x_segment,
    #         y=y_segment,
    #         mode='lines+markers',
    #         yaxis="y2",
    #         line=dict(color=color, width=2),
    #         showlegend=False,
    #         marker=dict(
    #             symbol='circle',  # Choose marker shape
    #             size=8,           # Set marker size
    #             color=color,      # Set marker color
    #             opacity=0.8,      # Set marker opacity
    #         )
    #     ))

    # # Add a "dummy" trace for the legend
    # fig.add_trace(go.Scatter(
    #     x=[None],  # No actual points
    #     y=[None],
    #     mode='lines',
    #     name='Bolus Insulin',
    #     line=dict(color='blue')  # Use one of the colors to represent the legend item
    # ))

    # Add third quartile (Q3) line
    fig.add_trace(go.Scatter(
        x=df['Time'],
        y=df_basal_insulin['Q3'],
        mode='lines',
        line=dict(color='orange', dash='dot'),
        fill='tonexty',  # This fills the area between the Q1 and Q3 lines
        fillcolor='rgba(255, 165, 0, 0.3)',  # Set fill color with transparency
        name='Q1 and Q3'  # Use same label for legend
    ))

    # Additional annotations
    fig.add_annotation(
        x=df.index,
        y=df['HypoEvent'],
        text="Hypo",  # Annotation text
        showarrow=True,
        arrowhead=4,
        ax=0,
        ay=-30
    ),

    # Update layout
    fig.update_layout(
        title='Summary Statistics by Time of Day',
        xaxis_title='Time of Day',
        yaxis_title='Value',
        yaxis2=dict(
            title='Bolus',
            overlaying='y',
            side='right',
            showgrid=False  # Hide grid for secondary y-axis if desired
        ),
        xaxis=dict(tickmode='array'),
        legend=dict(x=0.01, y=0.99)
    )

    return fig

# Define the layout of the app
app.layout = html.Div([
    html.H1("Time Series Summary Statistics"),
    dcc.Graph(figure=create_plot())  # Use the create_plot function to generate the figure
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

In [17]:
import plotly.graph_objects as go
import pandas as pd

# Sample data
data = {
    'Time': pd.date_range(start='2023-01-01', periods=10, freq='D'),
    'Value': [10, 15, 13, 17, 14, 18, 15, 20, 19, 21]
}
df = pd.DataFrame(data)

# Create a line plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['Time'],
    y=df['Value'],
    mode='lines+markers',
    name='Value'
))

# Add an annotation at a specific point (e.g., max value)
max_value = df['Value'].max()
max_date = df.loc[df['Value'] == max_value, 'Time'].iloc[0]

fig.add_annotation(
    x=max_date,  # x position for annotation
    y=max_value,  # y position for annotation
    text=f"Max Value: {max_value}",  # Annotation text
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40  # Offset the text to avoid overlap with data point
)

# Additional annotations
fig.add_annotation(
    x=df['Time'][2],
    y=df['Value'][2],
    text="Interesting point",  # Annotation text
    showarrow=True,
    arrowhead=4,
    ax=0,
    ay=-30
)

# Update layout for better display
fig.update_layout(
    title="Line Plot with Annotations",
    xaxis_title="Time",
    yaxis_title="Value"
)

fig.show()
